In [ ]:
import pandas as pd 
import os
import sys
import tkinter as tk
from tkinter import filedialog
import csv
import json


In [ ]:
root=tk.Tk()
root.withdraw()
file1=filedialog.askopenfilename(title="Browse the required file...")

In [ ]:
df=pd.read_csv(file1)

In [ ]:
df['event_data'].astype('str')
df['event_data']=df['event_data'].apply(json.loads)
df2=pd.json_normalize(df['event_data'])
df3=pd.concat([df.drop(['event_data'],axis=1),df2],axis=1)
del df,df2
df3["type"]=df3["type"].replace("apron_management.","",regex=True)
cols = []
count = 1
for column in df3.columns:
    if column == 'time':
        cols.append(f'time.{count}')
        count+=1
        continue
    cols.append(column)
df3.columns = cols
df3.drop('version', inplace=True, axis=1)

In [ ]:
###STAGE_CHANGES###
stage_changes=df3[(df3['process'] != "alarm_housekeeper") & (df3['type'] == "dgs_state_changes")& (df3['event_type'] == "SafedockStateChangeEvent")]
stage_changes['VDGS_state']=stage_changes.prev_safedock_state.str.cat(stage_changes.safedock_state,sep='_To_')
stage_cols=["id", "type", "time.2", "dgs_id", "ifplid", "ac_type", "apron_id", "stand_id", "VDGS_state","event_classification", "carrier", "flight_nr"]
stage_changes=stage_changes[stage_cols]


In [ ]:
###ALARMS###
filter_col = [col for col in df3 if col.startswith('alarm.')]
alarms=df3[(df3['type'] == "alarms")&(df3['event_src'] == "Safedock")& (df3['event_type'] == "AlarmOccurrencesEvent")]
alarm_cols=["id", "host", "process", "time.1", "type", "site", "time.2", "event_classification", "alarm", "event_src", "event_type", "location_info", "schedule_info"]
alarm_cols=alarm_cols+filter_col
alarms=alarms[alarm_cols]
alarms=alarms.explode('schedule_info')
alarms=alarms.explode('location_info')
alarm_error_list = ["ApronBlocked" , "AutoCalibrationError" ,"BridgeNotIn" , "GateBlocked","IDFailed - EngineCheckFailed" , "IDFailed - IdLostTrackCloseToStop","IDFailed - NoseHeightFailed" ,"IDFailed - VerAnalysisFailedTooClose", "SBUStop - AzimuthStop", "SBUStop - ScanFailed","SBUStop - ScanTooSlow" , "ViewBlocked"]
alarms=alarms[(alarms['alarm.alarm-description'].isin(alarm_error_list) )]
alarm_location=pd.json_normalize(alarms['location_info'])
alarm_location=alarm_location.add_prefix('location_info.')
alarm_schedule=pd.json_normalize(alarms['schedule_info'])
alarm_schedule=alarm_schedule.add_prefix('schedule_info.')
alarms=pd.concat([alarms.reset_index(),alarm_location.reset_index(),alarm_schedule.reset_index()],axis=1)
alarms['carrier_nr']=alarms['schedule_info.carrier'].str.cat(alarms['schedule_info.flight_nr'],sep=' ')
alarms=alarms[['id','time.1','type', 'site', 'time.2','event_classification','alarm.alarm-id', 'alarm.stand-id','alarm.alarm-code', 'alarm.alarm-description','carrier_nr']]
alarms.rename(columns = {'time.1':'time','time.2':'time.1','alarm.alarm-id':'alarm-id','alarm.stand-id':'stand-id','alarm.alarm-code':'alarm-code', 'alarm.alarm-description':'alarm-description'}, inplace = True)

In [ ]:
###sosFileSize###
sosFileSize=df3[(df3['type'] == "safedock_file_downloads")]
sos_columns=['id', 'time.1',  'stand_id','file_name','file_size','file_destination_path']
sosFileSize=sosFileSize[sos_columns]

In [ ]:
###aibt_aobt###
aibt_aobt=df3[(df3['type'] == "block_on_off")]
aibt_aobt['carrier_nr']=aibt_aobt['carrier'].str.cat(aibt_aobt['flight_nr'],sep=' ')
aibt_aobt_cols=["id", "type", "stand_id", "ac_type", "carrier_nr", "detected_aibt", "detected_aobt", "trigger_source"]
aibt_aobt=aibt_aobt[aibt_aobt_cols]

In [ ]:
###Commands###
commands=df3[((df3['type'] == "block_on_off")|(df3['type'] == "closed_for_maintenance")|(df3['type'] == "dgs_commands")|(df3['type'] == "dgs_state_changes"))&((df3['event_type'] != "AircraftScanEvent")&(df3['event_src'] != "SAM"))]
commands['carrier_nr']=commands['carrier'].str.cat(commands['flight_nr'],sep=' ')
commands=commands[~(commands['event_type'].isin(['EmergencyStopEvent','MaintenanceEvent','SafedockStateChangeEvent']))] 
commands=commands[(commands['event_type'].isin(['AbortDockingEvent','ParkOffEvent','ParkOnEvent','ShutdownScheduleEvent','StartDockingEvent']))] 
commands_columns=['id','time.2','event_type','stand_id','event_src','carrier_nr','ac_type']
commands=commands[commands_columns]
commands['time.2']=pd.to_datetime(commands['time.2'])
commands.rename(columns = {'time.2':'time'},inplace=True)


In [ ]:
###SAM_sequencing###
sam_sequencing=df3[(df3['type'] == "flight_updates")&(df3['flight_plan.fields.sequencing-state']=='Completed')&(df3['event_type']!='FlightTerminatedEvent')]
sam_sequencing['carrier_nr']=sam_sequencing['carrier'].str.cat(sam_sequencing['flight_nr'],sep=' ')
sam_sequencing['SAM_sequencing_state']=sam_sequencing['flight_plan_diff.fields.sequencing-state.from'].str.cat(sam_sequencing['flight_plan_diff.fields.sequencing-state.to'],sep='_To_')
# sam_filter_col = [col for col in sam_sequencing if col.startswith('flight_plan')]
sam_col=['id','flight_plan.timestamps.SEQT','flight_plan.fields.stand','flight_plan.fields.flight-plan-type','carrier_nr','flight_plan.fields.update-counter', 'ac_type','flight_plan.fields.actual-dgs-aircraft-type','flight_plan.fields.tow-in','SAM_sequencing_state','flight_plan.timestamps.EIBT','flight_plan.timestamps.DETECTED_AIBT','flight_plan.timestamps.EOBT','flight_plan.timestamps.DETECTED_AOBT']
# sam_col=sam_col+sam_filter_col
sam_sequencing=sam_sequencing[sam_col]
sam_sequencing['flight_plan.timestamps.SEQT']=pd.to_datetime(sam_sequencing['flight_plan.timestamps.SEQT'])
sam_sequencing.rename(columns={'flight_plan.timestamps.SEQT':'SEQT','flight_plan.fields.stand':'Stand','flight_plan.fields.flight-plan-type':'flight-plan-type','flight_plan.fields.update-counter':'update-counter','flight_plan.fields.actual-dgs-aircraft-type':'actual-dgs-aircraft-type','flight_plan.fields.tow-in':'tow-in','flight_plan.timestamps.EIBT':'EIBT','flight_plan.timestamps.DETECTED_AIBT':'DETECTED_AIBT','flight_plan.timestamps.EOBT':'EOBT','flight_plan.timestamps.DETECTED_AOBT':'DETECTED_AOBT'},inplace=True)

In [291]:
###Interface###
interface=df3[(df3['type'] == "equipment_state_changes") & df3['event_type'].isin(['GPUOffEvent','GPUOnEvent','PLBDockedEvent','PLBInEvent'])]
interface_col=["id","time.2","event_type","equipment_id"]
interface=interface[interface_col]
interface['time.2']=pd.to_datetime(interface['time.2'])
interface.rename(columns={"time.2":"time"})

,id,time.2,event_type,equipment_id
103,22750133,2022-04-24 02:35:25.480,PLBInEvent,PLBC55A
892,22750885,2022-04-24 03:00:22.512,GPUOffEvent,GPUC58
893,22750886,2022-04-24 03:00:22.520,PLBDockedEvent,PLBC58B
923,22750944,2022-04-24 03:02:42.105,PLBDockedEvent,PLBC58A
1391,22751367,2022-04-24 03:14:07.412,PLBDockedEvent,PLBC54B
...,...,...,...,...
45888,22791578,2022-04-24 23:56:32.661,PLBInEvent,PLBC53A
45893,22791592,2022-04-24 23:56:59.276,PLBDockedEvent,PLBF22A
45894,22791593,2022-04-24 23:56:59.282,GPUOffEvent,GPUF22
45895,22791594,2022-04-24 23:56:59.288,PLBDockedEvent,PLBF22B
